This notebook is licensed under the MIT License. See the [LICENSE file](https://github.com/tommasocarraro/LTNtorch/blob/main/LICENSE) in the project root for details.

# Grounding in Logic Tensor Networks (LTN)

## Real Logic

The semantics of LTN differs from the standard abstract semantics of First-order Logic (FOL). Specifically, LTN domains are interpreted concretely by tensors in the Real field.

To emphasize the fact that LTN interprets symbols which are grounded on real-valued features, we use the term *grounding*, denoted by $\mathcal{G}$, instead of interpretation.
$\mathcal{G}$ associates a tensor of real numbers to any term of the language, and a real number in the interval $[0,1]$ to any formula $\phi$.

In the rest of the tutorials, we commonly use "tensor" to designate "tensor in the Real field".

Real Logic language consists of a non-logical part (the signature) and logical connectives and quantifiers.
* **constants** denote individuals from some space of tensors $\bigcup\limits_{n_1 \dots n_d \in \mathbb{N}^*} \mathbb{R}^{n_1 \times \dots \times n_d}$ (tensor of any rank). The individuals can be pre-defined (data point) or learnable (embedding).
* **variables** denote sequences of individuals.
* **functions** can be any mathematical function either pre-defined or learnable. Examples of functions can be distance functions, regressors, etc. Functions can be defined using any operations in PyTorch. They can be linear functions, Deep Neural Networks, and so forth.
* **predicates** are represented as mathematical functions that map from some n-ary domain of individuals to a real number in $[0,1]$, which can be interpreted as a truth degree. Examples of predicates can be similarity measures, classifiers, etc.
* **connectives** ($\lnot, \land, \lor, \implies$) are modeled using fuzzy connective operators.
* **quantifiers** are defined using fuzzy aggregators.

This tutorial explains how to ground constants, variables, functions, and predicates.

In order to properly use LTN, we need to import the framework and PyTorch.

In [1]:
import ltn
import torch
import numpy as np

## Constants

LTN constants are grounded into real tensors. Each constant $c$ is mapped to a point in $\mathcal{G}(c) \in \bigcup\limits_{n_1 \dots n_d \in \mathbb{N}^*} \mathbb{R}^{n_1 \times \dots \times n_d}$. Notice that the objects in the domain may be tensors of any rank. For instance, a tensor of rank $0$ corresponds to a scalar, a tensor of rank $1$ to a vector, a tensor of rank $2$ to a matrix and so forth, in the usual way.

Here, we define $\mathcal{G}(c_1)=(2.1,3)$ and $\mathcal{G}(c_2)=\begin{pmatrix}
4.2 & 3 & 2.5\\
4 & -1.3 & 1.8
\end{pmatrix}$.

Note that the `Constant` constructor needs a PyTorch tensor containing the features of the individual.

In [2]:
c1 = ltn.Constant(torch.tensor([2.1, 3]))
c2 = ltn.Constant(torch.tensor([[4.2, 3, 2.5], [4, -1.3, 1.8]]))

Note that a constant can be set as learnable by using the optional argument `trainable=True`. This is useful to learn embeddings for some individuals.
The features of the tensor will be considered as trainable parameters (learning in LTN is explained in a further notebook).

In [4]:
c3 = ltn.Constant(torch.tensor([0.,0.]), trainable=True)

Under the hood, LTN constants are implemented using `LTNObject` objects. These particular objects wrap a value (the individual) and an important attribute called `free_vars`, which will be explained later in the tutorial.

If the parameter `trainable` of the LTN constant is set to `True`, then
the `requires_grad` parameter of the PyTorch tensor contained in the LTN constant will be set to `True`.

The value of an LTN constant can be easily accessed using the `value` attribute.

In [5]:
print(c1.value)
print(c3.value)
# here, we have to perform a detach before calling numpy(), because the tensor has requires_grad=True
print(c3.value.detach().numpy())

tensor([2.1000, 3.0000])
tensor([0., 0.], requires_grad=True)
[0. 0.]


## Predicates

LTN predicates are functions which map from some n-ary space of input values to a value in [0., 1.]. Predicates in LTN can be neural networks or any other function that achieves such a mapping.

There are different ways to construct a predicate in LTN. The constructor `ltn.Predicate(model, func)` permits two types of construction:
- if the `model` parameter is not `None`, the predicate is constructed by using the `torch.nn.Module` model instance that has been given as input;
- if the `model` parameter is `None` and the `func` parameter is not `None`, the predicate is constructed by using the function given in input. The construction by using
a function is suggested for small mathematical operations with **no weight tracking** (non-trainable functions).

The following defines a predicate $P_1$ using the `func` parameter and a predicate $P_2$ using the `model` parameter.

Notice the access to the `value` attribute for the constant `mu` inside the predicate $P_1$ definition. We need to do that because
`x` is a PyTorch tensor, while `mu` an LTN constant. The operand - is not supported for these two different types. In general, every time
an LTN object (constant or variable) is used inside the definition of a predicate, we need to access its value.

In [8]:
mu = ltn.Constant(torch.tensor([2., 3.]))
P1 = ltn.Predicate(func=lambda x: torch.exp(-torch.norm(x - mu.value, dim=1)))

class ModelP2(torch.nn.Module):
    """For more info on how to use torch.nn.Module:
    https://pytorch.org/docs/stable/generated/torch.nn.Module.html"""
    def __init__(self):
        super(ModelP2, self).__init__()
        self.elu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid()
        self.dense1 = torch.nn.Linear(2, 5)
        self.dense2 = torch.nn.Linear(5, 1) # returns one value in [0,1]

    def forward(self, x):
        x = self.elu(self.dense1(x))
        return self.sigmoid(self.dense2(x))

modelP2 = ModelP2()
P2 = ltn.Predicate(model=modelP2)

One can easily query predicates using LTN constants and LTN variables (see further in this notebook to query using variables).

Notice the access to the `value` attribute on the evaluation of the predicates. We need to do that because LTN predicates
return `LTNObject` instances. Inside the `value` attribute of the returned `LTNObject` there is the actual value of the predicate.

In [11]:
c1 = ltn.Constant(torch.tensor([2.1, 3]))
c2 = ltn.Constant(torch.tensor([4.5, 0.8]))
c3 = ltn.Constant(torch.tensor([3.0, 4.8]))
print(P1(c1).value)
print(P1(c2).value)
print(P2(c3).value)
print(P1(c1))

tensor(0.9048)
tensor(0.0358)
tensor(0.3344, grad_fn=<ViewBackward>)


NOTES:
- If an LTN predicate (or an LTN function) takes several inputs, e.g. $P_4(x_1,x_2)$, the arguments must be separated by a comma.
- LTN internally converts the inputs such that there is a "batch" dimension on the first dim. Therefore, most operations should work with `dim=1` (dimension of the features).

Here, we show the implementation of a binary predicate in LTN.

In [12]:
class ModelP4(torch.nn.Module):
    def __init__(self):
        super(ModelP4, self).__init__()
        self.elu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid()
        self.dense1 = torch.nn.Linear(4, 5)
        self.dense2 = torch.nn.Linear(5, 1) # returns one value in [0,1]

    def forward(self, x, y):
        x = torch.cat([x, y], dim=1)
        x = self.elu(self.dense1(x))
        return self.sigmoid(self.dense2(x))

P4 = ltn.Predicate(ModelP4())
c1 = ltn.Constant(torch.tensor([2.1, 3]))
c2 = ltn.Constant(torch.tensor([4.5, 0.8]))
print(P4(c1, c2).value) # multiple arguments are passed as a list

tensor(0.1631, grad_fn=<ViewBackward>)


For more details and useful ways to create predicates (incl. how to integrate multiclass classifiers as binary predicates) see the examples included in the /examples folder of this repository.

## Functions

LTN functions are grounded as any mathematical function that maps $n$ individuals to one individual in the tensor domain.

There are different ways to construct a function in LTN. The constructor `ltn.Function(model, func)` permits two types of construction:
- if the `model` parameter is not `None`, the function is constructed by using the `torch.nn.Module` model instance that has been given as input;
- if the `model` parameter is `None` and the `func` parameter is not `None`, the function is constructed by using the function given in input. The construction by using
a function is suggested for small mathematical operations with **no weight tracking** (non-trainable function).

The following defines a function $f_1$ using the `func` parameter, and a function $f_2$ using the `model` parameter.

In [13]:
f1 = ltn.Function(func=lambda x, y: x - y)

class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = torch.nn.Linear(2, 10)
        self.dense2 = torch.nn.Linear(10, 5)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2(x)

model_f2 = MyModel()
f2 = ltn.Function(model=model_f2)

One can easily compute functions using LTN constants and LTN variables as inputs (see further in this notebook to query using variables).

Notice that the output of an LTN function is an `LTNObject`, as we have already seen for the predicates.

The consideration about the necessity of accessing the `value` parameter of the returned `LTNObject` remains the same also for the LTN functions.

In [15]:
c1 = ltn.Constant(torch.tensor([2.1, 3]))
c2 = ltn.Constant(torch.tensor([4.5, 0.8]))
print(f1(c1, c2).value) # multiple arguments are passed as a list
print(f2(c1).value)
print(f2(c1))

tensor([-2.4000,  2.2000])
tensor([-0.5990,  0.0462,  0.9553,  0.2285, -0.1920], grad_fn=<ViewBackward>)


## Variables

LTN variables are sequences of individuals/constants from a domain. Variables are useful in logic to write quantified statements, e.g. $\forall x\ P(x)$.

Notice that a variable is a sequence and not a set, namely, the same value can occur twice in the sequence.

The following defines two variables, $x$ and $y$, with respectively 10 and 5 individuals sampled from normal distributions in $\mathbb{R}^2$.
In LTN, variables need to be labelled (see the arguments `'x'` and `'y'` below). The labels are used internally by LTN to recognize the variables and to
implement a lot of other important features.

The constructor internally builds an `LTNObject` for the variable. The value will be the given tensor, while the `free_vars` attribute
will be the list ['x'] for the first variable, and the list ['y'] for the second variable.

The interpretation of the `free_vars` attribute is that it contains the list of the labels of the free variables contained in the `LTNObject`.
In logic, a variable is free when it is not quantified by a quantifier, for example the existential quantifier or the universal quantifier.

In the case of a variable, it is intuitive to understand that the only free variable is the variable itself. It is for this reason that the `free_vars` attribute
contains only the label of the variable itself.

For the constants, we did not talk about their `free_vars` attribute since it is an empty list. It is intuitive since a constant
is not a variable, so it does not contain free variables.

In [17]:
x = ltn.Variable('x', torch.randn((10, 2)))
y = ltn.Variable('y', torch.randn((5, 2)))

Evaluating a term/predicate on one variable of $n$ individuals yields $n$ output values, where the $i$-th output value corresponds to the term calculated with the $i$-th individual.

Similarly, evaluating a term/predicate on $k$ variables $(x_1,\dots,x_k)$, with respectively $n_1,\dots,n_k$ individuals each, yields a result with $n_1 \times \dots \times n_k$ values. The result is organized in a tensor where the first $k$ dimensions can be indexed to retrieve the outcome(s) that correspond to each variable. The tensor is wrapped in an `LTNObject` where the attribute `free_vars` tells which dim of the tensor corresponds to which variable (using the label of the variable).

In the example below, the result of $P_4$ has shape 10x5 since x has 10 individuals and y 5 individuals.
The first cell of the result is the evaluation of $P_4$ on the first individual of x and first individual of y, the second
cell is the evaluation of $P_4$ on the first individual of x and second individual of y, and so forth.

Notice the use of the method `shape()` invoked on the `LTNObject` representing the evaluation of $P_4$. This method is a
short-cut for `res1.value.shape`.

The attribute `free_vars` contains in this case x and y. This is intuitive since no quantifier has been applied.

As shown on the last prints, it is possible to access single cells of the tensor containing the result, or the entire result.

In [19]:
# Notice that the outcome is a 2-dimensional tensor where each cell
# represents the satisfiability of P4 evaluated with each individual in x and in y.
P4 = ltn.Predicate(ModelP4())
res1 = P4(x, y)
print(res1.shape())
print(res1.free_vars) # dynamically added attribute; tells that axis 0 corresponds to x and axis 1 to y
print(res1.value[2, 0]) # gives the result computed with the 3rd individual in x and the 1st individual in y
print(res1.value)

torch.Size([10, 5])
['x', 'y']
tensor(0.5214, grad_fn=<SelectBackward>)
tensor([[0.4613, 0.4554, 0.4371, 0.4222, 0.4337],
        [0.5605, 0.5518, 0.5022, 0.4732, 0.4861],
        [0.5214, 0.5183, 0.4742, 0.4522, 0.4628],
        [0.6544, 0.6495, 0.5967, 0.5687, 0.5667],
        [0.6029, 0.5915, 0.5419, 0.5071, 0.5168],
        [0.5663, 0.5653, 0.5084, 0.4846, 0.4832],
        [0.5665, 0.5623, 0.5084, 0.4821, 0.4863],
        [0.5778, 0.5717, 0.5191, 0.4899, 0.4956],
        [0.5978, 0.5867, 0.5368, 0.5023, 0.5126],
        [0.5035, 0.5013, 0.4635, 0.4439, 0.4547]], grad_fn=<ViewBackward>)


Now, we see the same example but applied to an LTN function instead of an LTN predicate.

Notice that the output is no more in [0., 1.] since we have now applied a function.

Notice also the changes on the shape. In the case of the predicate we had shape 10x5, since the
result contained 10x5 truth values in [0., 1.]. Now, the function does not return a scalar. Instead,
it returns a real vector in $\mathbb{R}^2$. This explains the new shape 10x5x2. For each combination of the two variables
we have now a vector in $\mathbb{R}^2$ returned.

In [20]:
# Notice that the last axe(s) correspond to the dimensions of the outcome;
# here, f2 projects to outcomes in R^2, so the outcome has one additional axis of dimension 2.
# the output tensor has shape (10, 5, 2) because variable x has 10 individuals, y has 5 individuals, and f1 maps in R^2
res2 = f1(x, y)
print(res2.shape())
print(res2.free_vars)
print(res2.value[2,0]) # gives the result calculated with the 3rd individual in x and the 1st individual in y
print(res2.value)

torch.Size([10, 5, 2])
['x', 'y']
tensor([-1.5968, -0.1742])
tensor([[[-3.6696, -0.8967],
         [-2.7439, -1.3845],
         [-2.2959, -0.0168],
         [-1.1128,  0.0048],
         [-1.9483,  0.5959]],

        [[-1.5144, -0.8138],
         [-0.5887, -1.3016],
         [-0.1407,  0.0661],
         [ 1.0424,  0.0877],
         [ 0.2069,  0.6788]],

        [[-1.5968, -0.1742],
         [-0.6712, -0.6620],
         [-0.2232,  0.7057],
         [ 0.9599,  0.7273],
         [ 0.1244,  1.3184]],

        [[ 0.5213, -0.9353],
         [ 1.4469, -1.4232],
         [ 1.8950, -0.0555],
         [ 3.0781, -0.0339],
         [ 2.2426,  0.5572]],

        [[-1.2380, -1.4488],
         [-0.3124, -1.9366],
         [ 0.1356, -0.5689],
         [ 1.3187, -0.5473],
         [ 0.4832,  0.0438]],

        [[-0.2912,  0.0918],
         [ 0.6344, -0.3960],
         [ 1.0824,  0.9717],
         [ 2.2655,  0.9932],
         [ 1.4300,  1.5843]],

        [[-0.6577, -0.1816],
         [ 0.2680, -0.6694],

In this last example, we apply a predicate to a variable and a constant.

It is interesting to notice that the `free_vars` attribute contains now only y, since only variable y has been given in input.

Also, the shape is changed. Now, it is a vector of 5 values. The first cell contains the evaluation of the predicate on
$c_1$ and the first individual of $y$, the second cell contains the evaluation of the predicate on $c_1$ and the second individual of $y$, and so forth.
Since $y$ has 5 individuals in total, the result is intuitively a vector containing 5 truth values.

In [21]:
c1 = ltn.Constant(torch.tensor([2.1, 3]))
res3 = P4(c1, y)
print(res3.shape()) # Notice that no axis is associated to a constant. The output has shape (5,) because variable y has
# 5 individuals
print(res3.free_vars)
print(res3.value[0]) # gives the result calculated with c1 and the 1st individual in y
print(res3.value)

torch.Size([5])
['y']
tensor(0.4494, grad_fn=<SelectBackward>)
tensor([0.4494, 0.4598, 0.3997, 0.3894, 0.3824], grad_fn=<ViewBackward>)


### Variables made of trainable constants

In LTN, it is possible to define variables which individuals are trainable constants. This is really useful in embedding learning
tasks, where the individuals of a variable could be embeddings that have to be learned.

In order to better understand how trainable variables can be used in LTN, it is kindly suggested to carefully read the `Smokes-friends-cancer` example, where variables
made of trainable constants are used.

In this example, two trainable constants are declared. Then, an LTN variable is built using these constants.
The first individual of the variable will be the first constant, while the second individual will be the second constant.

Notice that after $P_2$ has been evaluated on $x$, both individuals of $x$ will have a `grad_fn` attribute. This means the gradient
has passed through the learnable constants by using the predicate.

It is important to note that in the declaration of the variables we have given the values of the constants and not the constants themselves.
This is why the two constants have `grad_fn` set to `None` after the predicate is applied. Since we have given the value, and
not the constant itself, the constants will remain untouched after the application of the predicate. Specifically, we have given the values of the constants to the variable since
LTN variables only accepts PyTorch tensors as values.

In [29]:
c1 = ltn.Constant(torch.tensor([2.1, 3]), trainable=True)
c2 = ltn.Constant(torch.tensor([4.5, 0.8]), trainable=True)

# PyTorch will keep track of the gradients between c1, c2 and x.
# Read tutorial 3 for more details.
x = ltn.Variable('x', torch.stack([c1.value, c2.value]))
res = P2(x)
print(res.value)

print(x.value[0])
print(x.value[1])

print(c1.value.grad_fn)
print(c2.value.grad_fn)

tensor([0.3727, 0.4806], grad_fn=<ViewBackward>)
tensor([2.1000, 3.0000], grad_fn=<SelectBackward>)
tensor([4.5000, 0.8000], grad_fn=<SelectBackward>)
None
None
